In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from utils import utils,myplot

from grid_world.data_parser import DataParser
from grid_world.grid_world import GridWorld

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv(r'wifi_track_data\dacang\track_data\dacang_track_data_5_clustered_0105.csv')
df.t = pd.to_datetime(df.t)
mac_list = df.m.unique()

#get path & pos data
df_wifipos = pd.read_csv('wifi_track_data/dacang/pos_data/wifi_pos_new_3_1220.csv')
df_path = pd.read_csv('wifi_track_data/dacang/pos_data/path_pos.csv')

%load_ext autoreload
%autoreload 2

## 初始化grid world

In [2]:
env_folder_path = r"wifi_track_data\dacang\grid_data\envs_grid\0130_40x30"
feature_folder_path = r"wifi_track_data\dacang\grid_data\features_grid\0130_40x30"
expert_traj_path = r"wifi_track_data\dacang\track_data\trajs_0117_40x30.csv"

world = GridWorld(environments_folderPath=env_folder_path,
                  features_folderPath=feature_folder_path,
                  expert_traj_filePath= expert_traj_path,
                  width=40,height=30)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from DMEIRL.value_iteration import value_iteration
import numpy as np
import os
from utils import utils

### Show Dynamics

In [5]:
world.dynamics.shape

(1200, 5, 1200)

In [6]:
world.ShowDynamics(0)

In [7]:
world.ShowDynamics(1)

In [8]:
world.ShowDynamics(2)

In [9]:
world.ShowDynamics(3)

In [10]:
world.ShowDynamics(4)

### Environments and Features

In [4]:
world.ShowEnvironments()

In [5]:
world.ShowFeatures()

### Tracks

In [13]:
world.ShowGridWorld_Count()

In [14]:
world.ShowGridWorld_Activated()

In [15]:
world.experts.ShowTraj(world.experts.mac_list[82])

In [16]:
df_cluster = pd.read_csv(r'wifi_track_data\dacang\cluster_data\cluster_result_0105.csv')
world.experts.ReadCluster(df_cluster)
world.experts.df_trajs_all

,m,trajs,cluster
0,"7.5-48,74,38,133,29,213","[[420, 1], [460, 0], [460, 3], [459, 0], [459,...",1
1,"7.5-48,74,38,155,214,98","[[420, 1], [460, 0], [460, 3], [459, 0], [459,...",1
2,"7.5-20,178,229,137,47,142","[[625, 0], [625, 3], [624, 1], [664, 0], [664,...",-1
3,"7.6-48,74,38,155,214,98","[[832, 3], [831, 3], [830, 0], [830, 3], [829,...",1
4,"7.6-32,50,51,210,96,61","[[832, 3], [831, 3], [830, 0], [830, 3], [829,...",1
...,...,...,...
1338,"8.7-252,4,28,35,119,153","[[788, 2], [748, 3], [747, 0], [747, 2], [707,...",-1
1339,"8.7-124,161,119,201,50,40","[[743, 1], [783, 4], [784, 0], [784, 0], [784,...",-1
1340,"8.7-56,251,20,50,26,70","[[910, 0], [910, 0], [910, 0], [910, 2], [870,...",0
1341,"8.7-144,173,247,200,14,93","[[832, 3], [831, 3], [830, 0], [830, 3], [829,...",0


In [9]:
world.experts.ApplyCluster((0,1))

In [10]:
world.experts.df_trajs

,m,trajs,cluster
0,"7.5-48,74,38,133,29,213","[[1641, 1], [1721, 1], [1801, 3], [1800, 0], [...",1
1,"7.5-48,74,38,155,214,98","[[1641, 1], [1721, 1], [1801, 3], [1800, 0], [...",1
2,"7.6-48,74,38,155,214,98","[[3344, 3], [3343, 3], [3342, 3], [3341, 0], [...",1
3,"7.6-32,50,51,210,96,61","[[3344, 3], [3343, 3], [3342, 3], [3341, 0], [...",1
4,"7.6-48,74,38,133,29,213","[[3344, 3], [3343, 3], [3342, 3], [3341, 0], [...",1
...,...,...,...
1013,"8.7-8,250,121,45,167,67","[[3743, 0], [3743, 0], [3743, 0], [3743, 0], [...",1
1014,"8.7-76,2,32,111,66,59","[[3097, 3], [3096, 2], [3016, 3], [3015, 3], [...",0
1015,"8.7-56,251,20,50,26,70","[[3660, 0], [3660, 2], [3580, 0], [3580, 0], [...",0
1016,"8.7-144,173,247,200,14,93","[[3344, 3], [3343, 3], [3342, 3], [3341, 0], [...",0
